In [1]:
from apis import GraphAPI
from spacymoji import Emoji
from pprint import pprint
import pandas as pd
import spacy
import time
import math
import re

In [20]:
#handles = ['itz_mohit_verma', 'gsakashsingh', 'iharinderkaur', 'makeupbypavani', 'beingmomtastic', 'yulia_aslamova', 'kabiakankhya', 'sindhucya', 'himanistyleunplug', 'pratikshabarii', 'anushabangia']
handles = ['isha_manju']

In [3]:
nlp = spacy.load('en_core_web_md')
emoji = Emoji(nlp)
nlp.add_pipe(emoji, first=True)

In [22]:
#profile_details_df = pd.DataFrame(columns= ['permalink', 'is_promotional', 'ORG_count', 'PRODUCT_count', 'total_NER', 'ner_list', 'propn', 'propn_list', 'caption_length', 'no_of_hashtags', 'no_of_emojis', 'standard_deviation'])
profile_details_df = pd.DataFrame()

def logreg(NER_count, propn_count, likes_count_stdev, captions_count_stdev):
    coef = [1.20112155, -0.0150792, 0.66137062, -0.66630538]
    bias = -0.82816386
    x = bias + (coef[0] * NER_count) + (coef[1] * propn_count) + (coef[2] * likes_count_stdev) + (coef[3] * captions_count_stdev)
    pred = 1 / (1 + math.exp(-x))
    return pred


def deviation(array):
    mu = max(array)
    l = len(array)
    ar = []
    for x in range(l):
        ar.append(math.sqrt((array[x]-mu)**2)/l)
    total = sum(ar)
    for x in range(l):
        if total != 0:
            ar[x] = round((ar[x]/total)*100, 2)
    return ar


def remove_emoji(string):
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F" # emoticons
                                u"\U0001F300-\U0001F5FF" # symbols & pictographs
                                u"\U0001F680-\U0001F6FF" # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', string)


def clean(caption):
    try:
        caption = remove_emoji(caption)
        doc = nlp(caption)
    except:
        caption = remove_emoji(caption)
        doc = nlp(caption)
    no_of_emojis = len(doc._.emoji)
    ner_org = 0
    ner_product = 0
    ner_list = []
    propn = 0
    propn_list = []
    
    hashtags_list = re.findall("#(\w+)", caption)
    caption = re.sub('\n',' ', caption)
    caption_list = caption.split(" ")
    
    for word in hashtags_list:
        try:
            word_ = '#' + str(word)
            caption_list.remove(word_)
        except:
            continue
    
    for word in caption_list:
        if word == '' or word == ',' or word == '.' or word == '_':
            caption_list.remove(word)
    
    length = len(caption_list)
    caption = ' '.join(caption_list)
    no_of_hashtags = len(hashtags_list)
    
    try:
        caption = remove_emoji(caption)
        doc = nlp(caption)
    except:
        caption = remove_emoji(caption)
        doc = nlp(caption)
        
    for entity in doc.ents:
        # pprint(entity.text + " : " + entity.label_ + " : " + str(spacy.explain(entity.label_)))
        if entity.label_ == 'ORG':
            ner_org += 1
            ner_list.append(entity)
        elif entity.label_ == 'PRODUCT':
            ner_product += 1
            ner_list.append(entity)
    
    try:
        caption_ = re.sub('#','',caption)
        #caption_ = re.sub('•','',caption)
        #caption_ = re.sub('-','',caption)
        #caption_ = re.sub('_','',caption)
        caption = remove_emoji(caption_)
    except:
        caption_ = re.sub('#','',caption)
        #caption_ = re.sub('•','',caption)
        #caption_ = re.sub('-','',caption)
        #caption_ = re.sub('_','',caption)
        caption = remove_emoji(caption_)

    for token in doc:
        if token.pos_ == 'PROPN':
            if token.text == '•' or token.text == '-' or token.text == '_':
                continue
            else:
                propn += 1
                propn_list.append(token)

    return length, no_of_hashtags, no_of_emojis, ner_org, ner_product, ner_list, propn, propn_list


x = []
count = 0
count_promotional = 0
promotional_posts_count = 0
for handle in handles:
    data = {
        'handle' : str(handle), 
        'global_stats' : {
            'total_caption_length' : '',
            'list_caption' : [],
            'total_likes': '',
            'list_likes' : [],
            'standard_deviation' : '',
            'total_hashtag_length' : '',
            'total_NER': '',
            'total_posts': ''
        },
        'posts' : []    
    }
    
    '''
    with open(f'C://Users//Ceeva//Desktop//Github//InfluencerScraper-master//handles//{handle}.json') as f:
        insta_user = json.load(f)
        # print(insta_user)
    '''
    insta_user = GraphAPI(handle)
    
    posts = insta_user['business_discovery']['media']['data']
    sum_length = 0
    sum_hashtags = 0
    sum_NER = 0
    sum_likes = 0
    list_likes = []
    list_captions = []
    list_NER_count = []
    list_PROPN_count = []
    promotional_posts_count = 0

    for post in posts:
        if 'caption' in post:
            length, no_of_hashtags, no_of_emojis, ner_org, ner_product, ner_list, propn, propn_list = clean(post['caption'])
            sum_length += length
            list_captions.append(length)
            sum_hashtags += no_of_hashtags
            sum_NER += ner_org
            sum_NER += ner_product
            sum_likes += post['like_count']
            list_likes.append(post['like_count'])

    data['global_stats']['total_caption_length'] = sum_length
    standard_deviation_captions = deviation(list_captions)
    data['global_stats']['list_captions'] = list_captions
    data['global_stats']['total_likes'] = sum_likes
    data['global_stats']['list_likes'] = list_likes
    standard_deviation_likes = deviation(list_likes)
    data['global_stats']['total_hashtag_length'] = sum_hashtags
    data['global_stats']['total_NER'] = sum_NER
    data['global_stats']['total_posts'] = len(posts)

    posts = insta_user['business_discovery']['media']['data']
    for post in posts:
        details = {
        'url': post['permalink'],
        'length': '',
        'hashtags': '',
        'like_count': '',
        'NER_list': [],
        'NER_count': '',
        'propn_count': '',
        'propn_list': [],
        'is_promotional': False,
        }
        if 'caption' in post:
            count_promotional += 1
            print(post['id'])
            length, no_of_hashtags, no_of_emojis, ner_org, ner_product, ner_list, propn, propn_list = clean(post['caption'])
            post_ner_sum = ner_org + ner_product
            list_NER_count.append(post_ner_sum)
            list_PROPN_count.append(propn)

            details['length'] = str(length)
            details['hashtags'] = str(no_of_hashtags)
            details['like_count'] = str(post['like_count'])
            details['NER_list'] = ner_list
            details['NER_count'] = post_ner_sum
            details['propn_count'] = str(propn)
            details['propn_list'] = propn_list
            if (ner_org + ner_product) > 1:
                details['is_promotional'] = True
                promotional_posts_count += 1
            x.append(details)
            pprint(details)
            
            #profile_details_df.loc[len(profile_details_df)] = [post['permalink'], details['is_promotional'], ner_org, ner_product, post_ner_sum, ner_list, propn, propn_list, length, no_of_hashtags, no_of_emojis]    
            profile_details_df = profile_details_df.append(details, ignore_index= True)
    profile_details_df['likes_count_stdev'] = standard_deviation_likes
    profile_details_df['captions_count_stdev'] = standard_deviation_captions

    logreg_predictions = []
    for index in range(len(standard_deviation_likes)):
        pred = logreg(list_NER_count[index], list_PROPN_count[index], standard_deviation_likes[index], standard_deviation_captions[index])
        logreg_predictions.append(pred)
    profile_details_df['predictions'] = logreg_predictions

print("TOTAL COUNT : " + str(count_promotional))
print("PROMOTIONAL POSTS : " + str(promotional_posts_count))
print(len(x))

print(profile_details_df.head(10))
profile_details_df.to_csv("./profile_details_df.csv")

17911714621695213
{'NER_count': 0,
 'NER_list': [],
 'hashtags': '0',
 'is_promotional': False,
 'length': '2',
 'like_count': '591',
 'propn_count': '0',
 'propn_list': [],
 'url': 'https://www.instagram.com/p/CNjlWT6hW_a/'}
17932204414513872
{'NER_count': 1,
 'NER_list': [Network18 Group],
 'hashtags': '2',
 'is_promotional': False,
 'length': '69',
 'like_count': '3119',
 'propn_count': '10',
 'propn_list': [World,
                Health,
                Day,
                CORONA,
                Network18,
                Group,
                Indians,
                Toh,
                aapne,
                LagayaKya],
 'url': 'https://www.instagram.com/p/CNXesyqhFk9/'}
17893416986002124
{'NER_count': 0,
 'NER_list': [],
 'hashtags': '0',
 'is_promotional': False,
 'length': '7',
 'like_count': '264',
 'propn_count': '0',
 'propn_list': [],
 'url': 'https://www.instagram.com/p/CNXRQH-hA6P/'}
17887938284033143
{'NER_count': 2,
 'NER_list': [Height adjustable headrest & harnes